In [1]:
import tensorflow as tf
import numpy as np
from IPython.display import clear_output, Image, display, HTML

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = tf.InteractiveSession()

# Helpers
Helper functions for TF Graph visualization


In [3]:

# Helper functions for TF Graph visualization

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:1000px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# CAMI Model as baseline (DM)

In [4]:
class CAMIModel(object):
    def __init__(self):
        self.settings = {}
        self.settings['filters_conv1'] = 6
        self.settings['k-max-pool1'] = 10
        self.settings['filters_conv2'] = 4
        self.settings['k-max-pool2'] = 4
    def buildModel(self, X, y, batch_size):

        assert X.shape[1:] == (20, 56)  # each event has 56 features and 20 phases.
        assert y.shape[1:] == 2

        # reshape for conv layer.
        X_reshaped = tf.reshape(X, shape=[-1, 20, 56, 1])

        # conv1
        conv1 = tf.layers.conv2d(inputs=X_reshaped, filters=self.settings['filters_conv1'],
                                 kernel_size=[20, 7], padding='valid', activation=tf.nn.tanh)
        tf.assert_equal(tf.shape(conv1)[1:], [1, 50, self.settings['filters_conv1']])  # (batch_size, 1, 50, 6)

        # transpose for kmax pooling
        T1 = tf.transpose(conv1, [0, 3, 1, 2])  # shape=(batch_size,6,1,50)

        # first kmax pooling
        k1 = self.settings['k-max-pool1']
        with tf.name_scope("kmaxpooling1"):
            pool1 = self.kmax_squish_pooling_general(T1, k1, batch_size)
            tf.assert_equal(tf.shape(pool1)[1:], [6, 1, k1]) #(batch_size, 6, 1, 10)

        # transpose for second conv layer
        pool1 = tf.transpose(pool1, [0, 2, 3, 1])  # shape=(batch_size,1,10,6)

        # conv2
        with tf.name_scope('conv2'):
            conv2 = tf.layers.conv2d(inputs=pool1, filters=self.settings['filters_conv2'],
                                     kernel_size=[1, 5], padding='valid', activation=tf.nn.tanh)
            tf.assert_equal(tf.shape(conv2)[1:], [1, k1 - 5 + 1, self.settings['filters_conv2']])  # (batch_size,1,6,4)

        # transpose for kmax pooling
        T2 = tf.transpose(conv2, [0, 3, 1, 2])  # shape=(batch_size,4,1,6)

        # second kmax pooling (hoi weird here!!!)
        k2 = self.settings['k-max-pool2']
        with tf.name_scope('k_max_pooling2'):
            pool2 = self.kmax_squish_pooling_general(T2, k2, batch_size)
            tf.assert_equal(pool2.get_shape()[1:], [4, 1, k2])  # (batch_size,4,1,4)

        pool2 = tf.transpose(pool2, [0, 2, 3, 1])  # shape=(batch_size,4,1,4)
        pool2 = tf.reshape(pool2, [-1, 1 * self.settings['filters_conv2'] * k2])

        # fully connect layer.
        fc = tf.layers.dense(pool2, units=2)
        y_hat = tf.nn.softmax(fc)
        # print(y_hat.get_shape())
        tf.assert_equal(y_hat.get_shape()[1:], [2])

        loss = -tf.reduce_mean(tf.reduce_sum(tf.multiply(y, tf.log(y_hat)), axis=1) )
        return loss, y_hat

    def fit(self):
        X = np.random.randn(1, 20, 56)
        y = np.random.randn(1,2)
        X_ph = tf.placeholder(tf.float32, shape=[None, 20, 56], name="X")
        y_ph = tf.placeholder(tf.float32, shape=[None, 2], name="y")
        bs_ph = tf.placeholder(tf.int32)
        loss, y_hat = self.buildModel(X_ph, y_ph, bs_ph)

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            lossVL = loss.eval(feed_dict={X_ph: X, y_ph: y, bs_ph: 1})
            print(lossVL)
    def visualize(self):
        lap_graph = tf.Graph()
        with lap_graph.as_default():
            X = np.random.randn(1, 20, 56)
            y = np.random.randn(1,2)
            X_ph = tf.placeholder(tf.float32, shape=[None, 20, 56], name="X")
            y_ph = tf.placeholder(tf.float32, shape=[None, 2], name="y")
            bs_ph = tf.placeholder(tf.int32, name="batch_size")
            loss, y_hat = self.buildModel(X_ph, y_ph, bs_ph)
        show_graph(lap_graph)
    def kmax_squish_pooling_general(self, inp, k, batch_size):
        '''
        only keep k largest values among the last dimension of inp. The order of elements must be kept same as before.
        Ex: if inp has shape (3,4,5) => result.shape = (3,4,k).
        '''
        assert inp.get_shape().ndims >= 2, "Expected tensor at least 2D but receive %sD" % (inp.get_shape().ndims)
        assert inp.get_shape()[-1] >= k, "k must less than or equal to the last dimension of the inputted tensor"
        values, indices = tf.nn.top_k(inp, k)
        # for some reasons, I have to negative indices to have indices sorted based their order
        T, _ = tf.nn.top_k(-indices, k)  # reverse order
        T = -T

        # we want to generate indices based on first ndims-1 dimensions of tensor @inp and the last dimension
        # is k (i.e. k largest values according to the last dimension)
        print(inp.get_shape())
        S = inp.get_shape()
        dimensions =  [tf.range(batch_size)] + [tf.range(S[1])] + [tf.range(e) for e in inp.get_shape()[2:-1]] + [tf.range(k)]
        # we swap the first two dimensions (It reduces readability of the code, but it works for meshgrid.), the rest dimensions
        # are kept same as before
        ls = [dimensions[1]] + [dimensions[0]] + dimensions[2:]
        # generate meshgrid
        mesh = tf.meshgrid(*ls)
        # again, we have to swap the first two dimension again. You may wonder why do I need to swap twice? Trust me, I tried
        # not to swap anything but it does not show the result I want.
        mesh = [mesh[1], mesh[0]] + mesh[2:]
        # after swap, we will stack the indices to first ndims-1 and dimsions found by tensor @T above.
        mesh = mesh[:-1] + [T]
        B = tf.stack(mesh, axis=-1)

        # time to collect easter eggs!!!
        r = tf.gather_nd(inp, B)
        return r

In [5]:
c = CAMIModel()
c.fit()

(?, 6, 1, 50)
(?, 4, 1, 6)
-3.7158847


In [6]:
c.visualize()

(?, 6, 1, 50)
(?, 4, 1, 6)


<img src='images/cami.png'></img>